# 環境設置

In [ ]:
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

!mkdir images
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
    
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

PyTorch version: 2.0.0+cu118
Torchvision version: 0.15.1+cu118
CUDA is available: False
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/base_command.py", line 160, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/req_command.py", line 241, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/pyt

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

# func

In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    polygons = []
    color = []
    for ann in sorted_anns:
        m = ann['segmentation']
        img = np.ones((m.shape[0], m.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:,:,i] = color_mask[i]
        ax.imshow(np.dstack((img, m*0.35)))

# 載入圖片

In [ ]:
image = cv2.imread('/content/ame_data/ame_1.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
plt.axis('off')
plt.show()

# Mask 模型設定

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

# Mask 參數設置 & output

In [ ]:
mask_generator = SamAutomaticMaskGenerator(
    model=sam,                
    points_per_side=10,           
    pred_iou_thresh=1,            
    stability_score_thresh=0.9,      
    crop_n_layers=1,            
    crop_n_points_downscale_factor=2,   
    min_mask_region_area=100,
)

Mask generation returns a list over masks, where each mask is a dictionary containing various data about the mask. These keys are:
* `segmentation` : the mask
* `area` : the area of the mask in pixels
* `bbox` : the boundary box of the mask in XYWH format
* `predicted_iou` : the model's own prediction for the quality of the mask
* `point_coords` : the sampled input point that generated this mask
* `stability_score` : an additional measure of mask quality
* `crop_box` : the crop of the image used to generate this mask in XYWH format

In [ ]:
masks = mask_generator.generate(image)

In [ ]:
# 顯示圖片 & all masks
plt.figure(figsize=(20,20))
plt.imshow(image)
show_anns(masks)
plt.axis('off')
plt.show()

# 以下為專案內容

In [ ]:
# 下載圖片
!gdown "1RPecIBIFfY5GL9g3Sz6TW3JobsBee7lC" --output ame.zip
!unzip ame.zip -d ame_data

In [ ]:
import glob

# 設定讀取資料夾
all_images = glob.glob("/content/ame_data/ame_*.jpg")

In [ ]:
index = 0

for img in all_images:
    img = cv2.imread(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 偵測
    img_masked = mask_generator_2.generate(img)

    # 區域
    x = int(img_masked[-1]["bbox"][0])
    y = int(img_masked[-1]["bbox"][1])
    x2 = int(img_masked[-1]["bbox"][0] + img_masked[0]["bbox"][2])
    y2 = int(img_masked[-1]["bbox"][1] + img_masked[0]["bbox"][3])

    img_cutted = img[int(y):int(y2), int(x):int(x2)]
    cv2.imwrite(f'/content/drive/MyDrive/Tibame/testing_files/cutted_img_{index}.jpg', img_cutted)

    index = index + 1